In [1]:
import os
import h5py
import shutil
import numpy as np

from pathlib import Path
from utils import phi_shift_and_flipping

MAX_CONSTI = {
    'Tower': 200,
    'Track': 100,
    'Lepton': 2,
}

# $\phi$ shifting and flipping

In [2]:
def to_event_image_h5(h5_path, out_h5):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['Tower/pt'][:], f_out['Track/pt'][:], f_out['Lepton/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['Tower/eta'][:], f_out['Track/eta'][:], f_out['Lepton/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['Tower/phi'][:], f_out['Track/phi'][:], f_out['Lepton/phi'][:]], axis=1)

        _, _, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['Tower/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['Track/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['Lepton/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

In [3]:
for name in ['ppZp-03', 'ppWW-03', 'ppZZ-03']:
    h5_path = f'./data/{name}.h5'
    out_h5 = f'./data/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/pre-processing/ppZp-03.h5
data/pre-processing/ppWW-03.h5
data/pre-processing/ppZZ-03.h5


# Pixelation

In [4]:
res = 40
h5_dir = Path('./data/pre-processing')
npy_dir = Path(f'./data/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['ppZp-03', 'ppWW-03', 'ppZZ-03']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/pre-processing/ppZp-03.h5 data/pre-processing/40x40/ppZp-03.npy 40 &
python pixelation.py data/pre-processing/ppWW-03.h5 data/pre-processing/40x40/ppWW-03.npy 40 &
python pixelation.py data/pre-processing/ppZZ-03.h5 data/pre-processing/40x40/ppZZ-03.npy 40 &
